<a href="https://colab.research.google.com/github/nicoleolivetto/CNN/blob/main/RNN_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("Women_clothing_reviews.csv").head(500)

In [6]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33.0,NaN,Absolutely wonderful - silky and sexy and comf...,4.0,1.0,0.0,Initmates,Intimate,Intimates
1,1,1080,34.0,NaN,Love this dress! it's sooo pretty. i happene...,5.0,1.0,4.0,General,Dresses,Dresses
2,2,1077,60.0,Some major design flaws,I had such high hopes for this dress and reall...,3.0,0.0,0.0,General,Dresses,Dresses
3,3,1049,50.0,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5.0,1.0,0.0,General Petite,Bottoms,Pants
4,4,847,47.0,Flattering shirt,This shirt is very flattering to all due to th...,5.0,1.0,6.0,General,Tops,Blouses


In [7]:
print(f'Shape of data: {df.shape}')
# Find the number of missing values
print(df.info())

Shape of data: (500, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               500 non-null    int64  
 1   Clothing ID              500 non-null    int64  
 2   Age                      500 non-null    float64
 3   Title                    403 non-null    object 
 4   Review Text              481 non-null    object 
 5   Rating                   500 non-null    float64
 6   Recommended IND          500 non-null    float64
 7   Positive Feedback Count  500 non-null    float64
 8   Division Name            500 non-null    object 
 9   Department Name          500 non-null    object 
 10  Class Name               500 non-null    object 
dtypes: float64(4), int64(2), object(5)
memory usage: 43.1+ KB
None


In [8]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Review Text'].astype(str).str.lower())

total_words = len(tokenizer.word_index)+1
tokenized_sentences = tokenizer.texts_to_sequences(df['Review Text'].astype(str))

In [9]:
# Slash sequences into n gram sequence
input_sequences = list()
for i in tokenized_sentences:
    for t in range(1, len(i)):
        n_gram_sequence = i[:t+1]
        input_sequences.append(n_gram_sequence)

# Pre pad with max sequence length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
# create predictors and labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [11]:
model = Sequential()
model.add(Embedding(total_words, 40, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(250)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='loss', min_delta=0, patience=4, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=10, verbose=1, callbacks=[earlystop])

Epoch 1/10
919/919 [==============================] - 66s 61ms/step - loss: 6.0109 - accuracy: 0.0575
Epoch 2/10
919/919 [==============================] - 21s 23ms/step - loss: 5.3458 - accuracy: 0.1106
Epoch 3/10
919/919 [==============================] - 20s 21ms/step - loss: 4.8977 - accuracy: 0.1419
Epoch 4/10
919/919 [==============================] - 18s 20ms/step - loss: 4.5264 - accuracy: 0.1631
Epoch 5/10
919/919 [==============================] - 17s 18ms/step - loss: 4.1506 - accuracy: 0.1886
Epoch 6/10
919/919 [==============================] - 18s 19ms/step - loss: 3.7701 - accuracy: 0.2162
Epoch 7/10
919/919 [==============================] - 19s 20ms/step - loss: 3.3919 - accuracy: 0.2620
Epoch 8/10
919/919 [==============================] - 17s 19ms/step - loss: 3.0362 - accuracy: 0.3189
Epoch 9/10
919/919 [==============================] - 18s 19ms/step - loss: 2.7036 - accuracy: 0.3810
Epoch 10/10
919/919 [==============================] - 18s 19ms/step - loss: 2.398